In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
def drawGraph(data):
    width = 20
    height = 20
    plt.figure(figsize=(width, height))
    dx = nx.DiGraph()

    dx.add_nodes_from(data['members'])
    dx.add_edges_from(data['edges'])
    pos = nx.spring_layout(dx)

    nx.draw_networkx_labels(dx, pos, data['labels'], font_size=16)
    nx.draw_networkx_nodes(dx, pos, nodelist=data['members'], node_color='r')
    nx.draw_networkx_nodes(
        dx, pos, nodelist=data['name'], node_color='y', node_shape='s', node_size=1300, linewidths=10)
    nx.draw_networkx_edges(
        dx, pos, edgelist=data['edges'], edge_color='b', arrows=True)
    plt.show()
    return

In [ ]:
def mergeDict(dictA, dictB):
    for m in dictB.keys():
        if m not in dictA.keys():
            dictA[m] = dictB[m]
        dictA[m] = (dictA[m])
    return dictA

def mergeData(data1, data2):
    allData = {}
    for i in data1.keys():
        if type(data1[i]) == type({}):
            allData[i] = mergeDict(data1[i], data2[i])
        else:
            allData[i] = data1[i] | data2[i]
            allData[i] = (allData[i])
    return allData

def getData(sentence):
    leftMembers = sentence['leftMembers']
    name = sentence['name']
    rightMembers = sentence['rightMembers']
    labels = {}
    edges = []
    for m in leftMembers:
        labels[m] = m
        edges.append((m, name[0]))
    for m in rightMembers:
        labels[m] = m
        edges.append((name[0], m))
    for m in name:
        labels[m] = m

    return ({'labels':  (labels), 'edges': set(edges),
             'members': set(leftMembers) | set(rightMembers), 'name': set(name)})

In [ ]:
import pydot
def drawGraph2(data):
    graph = pydot.Dot(graph_type='graph')
    pyNodes = {}
    print(data)
    for n in set(data['members']):
        pyNodes[n] = pydot.Node(n, fillcolor='r')
    for n in set(data['names']):
        pyNodes[n] = pydot.Node(n, fillcolor="b", shape='box')
    for edge in set(data['edges']):
        graph.add_edge(pydot.Edge(edge[0], edge[1], label='a', color='r'))
    graph.write_png('ojj.png')

In [ ]:
def findIndependetMembers(data):
    independentMember = {key: 0 for key in (set(data['members'])|set(data['name']))}
    for k in data['edges']:
        independentMember[k[1]] += 1
    return [l for l in independentMember.keys() if independentMember[l]==0]

def getTrues(data):
    return (set(data['members']) -  set([m for m in data['members'] for e in data['edges'] if (e[1]==m or e[0] == m)]))

def deleteSentences(data):
    toDelete = findIndependetMembers(data)
    for e in [l for l in data['edges'] if l[0] in toDelete]:
        data['edges'].remove(e)
    for e in toDelete:
        del data['labels'][e]
        data['name'].remove(e)
    trues = getTrues(data)
    for e in trues:
        data['members'].remove(e)
        del data['labels'][e]
    return data , trues

In [ ]:
originalSentences = [
    {'leftMembers': [], 'name': ('1'), 'rightMembers': ['a', 'b']},
    {'leftMembers': ['b'], 'name': ('2'), 'rightMembers': ['a']},
    {'leftMembers': [], 'name': ('3'), 'rightMembers': ['a', 'c']},
    {'leftMembers': ['a'], 'name': ('4'), 'rightMembers': ['d', 'e', 'f']},
    {'leftMembers': ['e'], 'name': ('5'), 'rightMembers': ['f']},
    {'leftMembers': ['f'], 'name': ('6'), 'rightMembers': ['e']},

]

trues = []
data = getData(originalSentences[0]);
for oS in originalSentences:
    data = mergeData(data, getData(oS))

#drawGraph2(data)
drawGraph(data)
data, deletedMembers = deleteSentences(data)

drawGraph(data)    
    